## Tutorial - Georastertools

Georastertools provides satellite image processing functions written in Python, based on *rasterio* and *geopandas* libraries. It includes a CLI (Command Line Interface) and a Python API that we will introduce in this tutorial.

Georastertools gives access to the following tools:

- **Radioindice** to compute radiometric indices such as NDVI, NDWI, NDBI, etc.
- **Timeseries** to generate a temporal sequence by computing temporal derivatives between two images
- **Tilling** to split an image into tiles based on a given geometry
- **Zonalstats** to compute zonal statistics on a specified geometries
- **Filter** to apply filters such as median, sum, mean, and adaptive gaussian
- **Hillshade** to simulate cast shadows for a given sun elevation


### Installation 

To install Georastertools, you can use the following commands in your terminal :

```bash
conda create -n env_name
conda activate env_name
conda install python=3.8.13 libgdal=3.5.0
pip install georastertools --no-binary rasterio
```

This will create a conda environment in your machine and install the required versions of *python* and *libgdal*. It then install *georastertools* and *rasterio* with the *--no-binary* option.

### Python API

The Python API of Georastertools allows to activate each tool and to extend their functionalities. The methods are all named with the following pattern *with_\<something\>* and return the current instance so that the methods can be chained. The method *eolab.rastertools.Rastertool.process_files* allows to process several files can be processed at the same time

Here, we present a slightly more advanced example, we define our own radiometric index and compute it with the Radioindice class. 

#### Exemple : Compute a user-defined indice

In [50]:
from eolab.rastertools import Radioindice
from eolab.rastertools.product import BandChannel
from eolab.rastertools.processing import RadioindiceProcessing
import numpy as np

We define the Normalized Difference Snow Index (NDSI) in a function that takes a numpy.ndarray as input and returns the computed NDSI in a numpy.ndarray.

The Normalized Difference Snow Index (NDSI) is calculated using the following formula:

$$\text{NDSI} = \frac{\text{Green Band} - \text{SWIR Band}}{\text{Green Band} + \text{SWIR Band}}$$

Then, we create a RadioindiceProcessing object and specify its input channels (i.e. the bands used to compute the NDSI).

In [47]:
def ndsi(bands : np.ndarray) -> np.ndarray :
    """
    Computes the NDSI of the input image
    """
    ndsi =  (bands[0] - bands[1]) / (bands[0] + bands[1])

    return ndsi


ndsi = RadioindiceProcessing("ndsi").with_channels([BandChannel.green, BandChannel.swir])

We then configure the output directory and our region of interest. The line *outputs = tool.process_files(inputfiles)* compute the NDSI of the given images and saves the output, please ensure that the output directory already exists in your machine.

In [51]:
inputfiles = ["data/SENTINEL2A_20180928-105515-685_L2A_T30TYP_D.zip","data/SENTINEL2B_20181023-105107-455_L2A_T30TYP_D.zip"]


tool = Radioindice([ndsi])
tool.with_output("output_radi", merge=True)
tool.with_roi("data/COMMUNE_32001.shp")
outputs = tool.process_files(inputfiles)

ndsi: 100%|███████████████████████████████████████| 4/4 [00:00<00:00, 16.20it/s]


### Command Line Interface

The CLI of Georastertools enables users to use the differents tools directly from the terminal without needing to write additional code. The generated files are automatically dowloaded on your machine.

The following command lists all the options that can be given to the CLI of Georastertools.

In [8]:
%%bash
rio georastertools --help

Usage: rio georastertools [OPTIONS] COMMAND [ARGS]...

  Main entry point for the `georastertools` Command Line Interface.

  The `georastertools` CLI provides tools for raster processing and analysis
  and allows configurable data handling, parallel processing, and debugging
  support.

  Logging:

      - INFO level (`-v`) gives detailed step information.

      - DEBUG level (`-vv`) offers full debug-level tracing.

  Environment Variables:

      - `RASTERTOOLS_NOTQDM`: If the log level is above INFO, sets this to
      disable progress bars.

      - `RASTERTOOLS_MAXWORKERS`: If `max_workers` is set, it defines the max
      workers for georastertools.

Options:
  -t, --rastertype PATH  JSON file defining additional raster types of input
                         files
  --max_workers INTEGER  Maximum number of workers for parallel processing. If
                         not given, it will default to the number of
                         processors on the machine. When all process

#### Exemple 1 : Compute zonal statistics

The zonalstats tool computes zonal statistics for a given input raster. The statistics are computed inside zones defined by a vector file, or, on the entire raster if no vector file is provided. It is also possible to generate charts to visualize the computed statistics.

The tool supports the computation of various statistics, including:
- Minimum and maximum
- Range
- Mean and standard deviation
- Percentile
- Median and Median absolute deviation
- Count, for all pixels, no data and valid pixels
- Sum
- Majority and minority, most and least frequent value
- Unique values
- ...
                              
To generate the zonalstatistics of a raster, we can provide different parameters that can be displayed with the following command :

In [7]:
%%bash
rio georastertools zonalstats --help

Usage: rio georastertools zonalstats [OPTIONS] INPUTS...

  Compute zonal statistics of a raster image.

  Available statistics are: min, max, range, mean, std, percentile_x (x in [0,
  100]), median, mad, count, valid, nodata, sum, majority, minority, unique.

  By default, only the first band is computed unless specified otherwise.

  Arguments:

      inputs TEXT

      Raster files to process. You can provide a single filewith extension
      ".lst" (e.g. "zonalstats.lst") that lists the input files to process
      (one input file per line in .lst)

Options:
  -o, --output TEXT           Output directory to store results (by default
                              current directory)
  -f, --format TEXT           Output format of the results when input
                              geometries are provided (by default ESRI
                              Shapefile). Possible values are ESRI Shapefile,
                              GeoJSON, CSV, GPKG, GML
  -g, --geometry TEXT         Li

The following example computes the statistics of the blue band of a Sentinel-2B image. The vector file that contains the statistical values calculated for each geometry is automatically saved in the "output_zonalstats" directory.

In [26]:
%%bash
rio georastertools zs -o output_zonalstats -f GeoJSON -g data/COMMUNE_32xxx.geojson --stats min --stats max --stats mean --stats std data/S2B_T30TYP_20241021T105009_B02_20m.jp2

To access the computed statistics, it is possible to open the generated file on QGIS. Then, open its attribute table by clicking on the icon : <img src="img/icon.png" alt="" width=30/>

The following table contains the results obtained:


<div style="text-align: center;">
    <img src="img/table_zs_full.png" alt="" width="1000"/>
</div>

If the provided input image does not cover the entire vector file, statistics can be computed only for the geometries that are fully covered by the image by adding the --within option. This option ensure that the statistics obtained cover the whole region and are accurate. 

<div style="text-align: center;">
    <img src="img/within_zs_geom.png" alt="" width="400"/>
</div>

In [24]:
%%bash
rio georastertools zs -o output_zonalstats -f GeoJSON -g data/COMMUNE_32xxx.geojson --stats min --stats max --stats mean --stats std --within data/SENTINEL2A_20180928-105515-685_L2A_T30TYP_D-ndvi.tif

The generated table now contains the statististics of only the two geometries that are contained in the image.

<div style="text-align: center;">
    <img src="img/within_zs_table.png" alt="" width="1000"/>
</div>

To compute the number of pixels corresponding to specific values, for example in a classified image, the *--categorical* option can be used.

In the following exemple we use zonalstats on a classification of Geneva from the 21-10-2023, with labels indicating clouds, water, land, and cloud shadows.

In [13]:
%%bash
rio georastertools zs -o output_zonalstats -f GeoJSON --categorical data/labeled_img_regular.tif

We obtain the following table, that contains the occurences of each class in the input image.

<div style="text-align: center;">
    <img src="img/zs_cat_table.png" alt="" width="600"/>
</div>


#### Exemple 2 : Compute a time series

Georastertools can generate time series between two raster images. The sequence of rasters is created using linear interpolation of every pixel of the input images.

If one of the inputs contains missing data, the interpolation enables to fill the gaps in all the rasters of the sequence.


If the start (or end) dates are before (or after) the dates of the input rasters, the corresponding generated raster will contain the values of the first (or last) input raster and may contain the same gaps as the input raster.


To generate a timeseries, we can provide different parameters that can be displayed with the following command : 

In [20]:
%%bash
rio georastertools timeseries --help

Usage: rio georastertools timeseries [OPTIONS] INPUTS...

  Generate a timeseries of images (without gaps) from a set of input images.
  Data not present in the input images (e.g., missing images for specific
  dates or masked data) are interpolated (with linear interpolation) so that
  all gaps in the timeseries are filled.

  This command is useful for generating continuous timeseries data, even when
  some input images are missing or contain masked values.

  Arguments:

  inputs TEXT

      Input file to process (e.g. Sentinel2 L2A MAJA from THEIA). You can
      provide a single file with extension ".lst" (e.g. "speed.lst") that
      lists the input files to process (one input file per line in .lst).

Options:
  -o, --output TEXT           Output directory to store results (by default
                              current directory)
  -s, --start_date TEXT       Start date of the timeseries to generate in the
                              following format: yyyy-MM-dd
  -e, --end_

In the following example, we compute the time series between two Sentinel-2 images from 2018-09-28 to 2018-10-23. The step between two consecutive images is 10 days and the images are processed using a window size of 512 x 512 pixels.

The processed files are automatically saved in the "output_timeseries" directory.

At least two input rasters must be provided, but additional rasters can be included if required.

In [6]:
%%bash
rio georastertools timeseries data/SENTINEL2A_20180928-105515-685_L2A_T30TYP_D-ndvi.tif data/SENTINEL2B_20181023-105107-455_L2A_T30TYP_D-ndvi.tif -o output_timeseries -s 2018-09-28 -e 2018-10-23 -p 10 -ws 512